# Preprocessing

In [7]:
import pandas

df = pandas.read_csv('s&p500.csv')
df = df.set_index('Date')
df

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-31 00:00:00+00:00,36.393887,39.179268,23.973495,41.956585,36.026073,18.773333,68.768227,70.120003,40.179710,35.525333,...,91.401855,22.913128,162.714340,24.432884,62.614067,32.120022,42.933430,99.750992,73.750000,34.574112
2014-11-03 00:00:00+00:00,37.588364,39.795147,24.284258,41.817753,35.778130,18.799999,68.641075,69.910004,40.260685,35.714283,...,90.792976,23.197344,162.286133,24.637280,61.675262,31.837324,42.772045,99.006737,73.879997,35.163273
2014-11-04 00:00:00+00:00,36.980907,40.458393,24.106678,41.427670,36.042606,18.910000,68.793671,71.070000,40.042057,37.445198,...,90.725319,23.319159,158.646683,24.432884,61.196133,31.748985,43.070908,97.930672,72.779999,36.468380
2014-11-05 00:00:00+00:00,36.934883,40.032017,24.164398,41.348339,36.083923,18.906668,69.921150,71.370003,40.511711,38.027210,...,92.010750,23.265015,153.919647,24.885473,61.552242,32.146519,43.298035,99.526817,73.070000,36.608208
2014-11-06 00:00:00+00:00,38.076160,40.875298,24.233505,41.460724,36.108704,19.026667,70.565430,72.099998,40.665558,37.770222,...,93.950172,23.170267,153.037628,24.126284,62.322708,32.782562,43.692513,98.809441,73.879997,37.018387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25 00:00:00+00:00,130.190002,13.150000,231.410004,187.850006,114.220001,105.300003,360.799988,483.720001,230.169998,56.560001,...,290.450012,31.799999,97.989998,64.459999,119.489998,130.419998,133.039993,102.349998,360.089996,180.009995
2024-10-28 00:00:00+00:00,131.539993,13.600000,233.399994,189.679993,114.070000,105.660004,361.320007,481.040009,230.139999,57.240002,...,292.119995,31.950001,98.620003,64.480003,118.900002,130.789993,134.860001,103.599998,363.579987,182.759995
2024-10-29 00:00:00+00:00,131.229996,13.820000,233.669998,189.449997,113.400002,104.800003,363.040009,485.390015,235.229996,56.320000,...,292.179993,30.879999,99.000000,63.340000,117.279999,129.889999,133.970001,103.879997,384.679993,181.270004


In [12]:
import torch

def preprocess(df):
    data = torch.from_numpy(df.to_numpy()).to(torch.float32)
    ratios = torch.zeros_like(data)
    # calculate daily return ratio
    for d in range(1, data.size(0)):
        ratios[d-1] = (data[d] - data[d-1]) / data[d-1]
    # skip the first day which cannot calculate daily return ratio
    # and round data size to nearest multiple of batch_size
    ratios = ratios[:2510]
    # split into batches
    ratios = ratios.view(251, 10, 472)
    return ratios

train_data = preprocess(df)
print(train_data.size())
print(train_data[0])

torch.Size([251, 10, 472])
tensor([[ 0.0328,  0.0157,  0.0130,  ..., -0.0075,  0.0018,  0.0170],
        [-0.0162,  0.0167, -0.0073,  ..., -0.0109, -0.0149,  0.0371],
        [-0.0012, -0.0105,  0.0024,  ...,  0.0163,  0.0040,  0.0038],
        ...,
        [-0.0050, -0.0002,  0.0141,  ..., -0.0018, -0.0047, -0.0247],
        [ 0.0000,  0.0159,  0.0141,  ..., -0.0023, -0.0184,  0.0141],
        [ 0.0029, -0.0025,  0.0121,  ..., -0.0044,  0.0011, -0.0023]])


# Model

In [13]:
import torch.nn as nn
import math

# https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, hidden_size, expand_ratio, dropout):
        super(FeedForward, self).__init__()
        self.linear = nn.Linear(hidden_size, hidden_size * expand_ratio)
        self.linear2 = nn.Linear(hidden_size * expand_ratio, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        return x

class Attention(nn.Module):
    def __init__(self, d_model, num_heads, expand_ratio, dropout):
        super().__init__()
        # self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(hidden_size=d_model, expand_ratio=expand_ratio, dropout=dropout)

    def forward(self, x, attn_mask=None):
        x = torch.randn(x.size()).to(torch.float32)
        # x1, _ = self.mha(x, x, x, attn_mask=attn_mask)
        x1 = self.mha(x, x, x, mask=attn_mask)
        x2 = self.ln1(x + x1)
        return self.ln2(self.ffn(x2) + x2)

class SpatialTemporalAttention(nn.Module):
    def __init__(self, d_model=64, num_heads=2, expand_ratio=4, dropout=0.1, T=9, N=493):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.input_proj = nn.Linear(1, d_model)
        self.time_embedding = nn.Embedding(T, d_model)
        self.stock_embedding = nn.Embedding(N, d_model)
        self.spatial_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.temporal_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.output_proj = nn.Linear(d_model, 1)
    
    def forward(self, x):
        T, N = x.size()
        # nn.MultiheadAttention returns all NaNs if stock_embedding is not added for some reason
        x = self.input_proj(x.view(T, N, 1)) + self.stock_embedding(torch.arange(N).unsqueeze(0).expand(T, N))
        # IDEA: Each spatial head takes in a different type of correlation matrix.
        # Like one takes in positive pearson's coefficnet and the other takes in negative
        x = self.spatial_attn(x).view(N, T, self.d_model)
        x = self.time_embedding(torch.arange(T).unsqueeze(0).expand(N, T)) + x
        # temporal_causal_mask = torch.tril(torch.ones((T, T), dtype=torch.bool)).expand(N * self.num_heads, T, T).to(x.device)
        temporal_causal_mask = torch.tril(torch.ones((T, T), dtype=torch.bool)).to(x.device)
        x = self.temporal_attn(x, attn_mask=temporal_causal_mask)
        return self.output_proj(x).view(T, N)


# Training

In [16]:
import numpy as np

# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cpu')

model = SpatialTemporalAttention().to(device)
train_data = train_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.MSELoss()

model.train()

num_epochs = 1

for epoch in range(num_epochs):
    optimizer.zero_grad()
    for batch in train_data:
        out = model(batch[:-1, :])
        loss = loss_fn(out, batch[1:, :])
        print(loss.item())
        loss.backward()
        optimizer.step()

0.38484862446784973
0.14685313403606415
1.1739323139190674
0.13534411787986755
0.5477238893508911
1.392273187637329
1.1198668479919434
0.4801308512687683
0.07905121147632599
0.04329857602715492
0.3182877004146576
0.8580579161643982
1.6119695901870728
2.3113129138946533
2.5681254863739014
2.310959577560425
1.795682668685913
1.3313757181167603
1.3815085887908936
2.12030029296875
3.115220069885254
3.9884867668151855
4.265028953552246
3.975684881210327
3.359455108642578
2.6473076343536377
2.0469987392425537
1.4802815914154053
1.037236213684082
0.6641921401023865
0.4295552968978882
0.22450487315654755
0.11619696021080017
0.06516369432210922
0.07770290970802307
0.1473153978586197
0.24228514730930328
0.37751346826553345
0.5173283815383911
0.6350460648536682
0.7409893274307251
0.803149402141571
0.7656819224357605
0.7352030277252197
0.6227862238883972
0.5331774950027466
0.48892906308174133
0.6172346472740173
0.8648272156715393
1.4220306873321533
2.392573118209839
3.3966121673583984
4.7879467010